In [2]:
import pandas as pd 

df = pd.read_csv('/Users/yudahendriawan/Library/Mobile Documents/com~apple~CloudDocs/Dicoding/google-colab-projects/datasets/imdb_indonesian_movies_2.csv')
df = df.drop(columns=['judul_film'])

In [3]:
df.head()

,ringkasan_sinopsis,genre
0,Raden Mas Said putra sulung Tumenggung Wilarik...,Drama
1,Soe Hok Gie adalah seorang aktivis yang hidup ...,Drama
2,Guru Bangsa Tjokroaminoto menceritakan tentang...,Drama
3,POL menceritakan kisah hidup yang luar biasa d...,Drama
4,Perjalanan pahlawan Indonesia KH Ahmad Dahlan ...,Drama


In [11]:
import numpy as np

category = pd.get_dummies(df.genre, dtype=np.float32)
df_baru = pd.concat([df, category], axis=1)
df_baru = df_baru.drop(columns='genre')
df_baru

,ringkasan_sinopsis,Drama,Horor,Komedi,Laga,Romantis
0,Raden Mas Said putra sulung Tumenggung Wilarik...,1.0,0.0,0.0,0.0,0.0
1,Soe Hok Gie adalah seorang aktivis yang hidup ...,1.0,0.0,0.0,0.0,0.0
2,Guru Bangsa Tjokroaminoto menceritakan tentang...,1.0,0.0,0.0,0.0,0.0
3,POL menceritakan kisah hidup yang luar biasa d...,1.0,0.0,0.0,0.0,0.0
4,Perjalanan pahlawan Indonesia KH Ahmad Dahlan ...,1.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...
1000,Winter in Tokyo berpusat pada kehidupan Ishida...,0.0,0.0,0.0,0.0,1.0
1001,Markonah melarikan diri ke Jakarta karena akan...,0.0,0.0,0.0,0.0,1.0
1002,"Tempat aking lebih dari 36 jam, Last Night ada...",0.0,0.0,0.0,0.0,1.0
1003,Proyek baru ini adalah tentang seorang lelaki ...,0.0,0.0,0.0,0.0,1.0


In [12]:
sinopsis = df_baru['ringkasan_sinopsis'].values
label = df_baru[['Drama','Horor', 'Komedi','Laga','Romantis']].values
sinopsis

array(['Raden Mas Said putra sulung Tumenggung Wilarikta di bawah Kerajaan Majapahit yang berkuasa di wilayah Tuban melihat sekeluarga miskin yang menderita busung lapar. Ia merasa sangat prihatin dan hati nuraninya tergugah untuk menolong. Kemudian ia mencoba secara diam-diam mengambil makanan dari lumbung orang tuanya. Perbuatan itu tidak disetujui orang tuanya bahkan ia dihukum sekap di gudang makanan itu. Sejak kejadian itu RM Said yang tumbuh dewasa tidak betah tinggal di rumah. Ia berkelana dari daerah satu ke daerah lainnya. Dari sanalah ia tahu betapa banyak penyelewengan dan kesewenang-wenangan para lurah yang munafik. Mereka selalu mengkambing-hitamkan Tumenggung untuk menutupi kejahatannya. Atas laporan RM Said ayahnya kemudian sadar. Tetapi kemudian ia dianggap sebagai sumber fitnah. Dalam kelananya kemudian ia bertemu dengan Sunan Bonang yang banyak mencurahkan ilmunya kepada RM Said. Ia pun kemudian melakukan tapa di pinggir kali. Berkat ketabahannya menghadapi berbagai c

In [13]:
label

array([[1., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0.],
       ...,
       [0., 0., 0., 0., 1.],
       [0., 0., 0., 0., 1.],
       [0., 0., 0., 0., 1.]], dtype=float32)

In [15]:
from sklearn.model_selection import train_test_split

sinopsis_latih, sinopsis_test, label_latih, label_test = train_test_split(sinopsis, label, test_size=0.2)

In [17]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

tokenizer = Tokenizer(num_words=5000, oov_token='x')
tokenizer.fit_on_texts(sinopsis_latih)
tokenizer.fit_on_texts(sinopsis_test)

sekuens_latih = tokenizer.texts_to_sequences(sinopsis_latih)
sekuens_test = tokenizer.texts_to_sequences(sinopsis_test)

padded_latih = pad_sequences(sekuens_latih)
padded_test = pad_sequences(sekuens_test)

In [19]:
import tensorflow as tf 

model = tf.keras.Sequential(
    [
        tf.keras.layers.Embedding(input_dim=5000, output_dim=16),
        tf.keras.layers.LSTM(64),
        tf.keras.layers.Dense(128, activation='relu'),
        tf.keras.layers.Dense(64, activation='relu'),
        tf.keras.layers.Dense(5, activation='softmax')
    ]
)
model.compile(
    loss='categorical_crossentropy',
    optimizer='adam',
    metrics=['accuracy']
)

In [20]:
num_epochs = 30
history = model.fit(
    padded_latih,
    label_latih,
    epochs=num_epochs,
    validation_data=(padded_test, label_test),
    verbose=2
)

Epoch 1/30
26/26 - 11s - loss: 1.6099 - accuracy: 0.1915 - val_loss: 1.6102 - val_accuracy: 0.1990 - 11s/epoch - 421ms/step
Epoch 2/30
26/26 - 10s - loss: 1.6055 - accuracy: 0.2326 - val_loss: 1.6064 - val_accuracy: 0.2537 - 10s/epoch - 385ms/step
Epoch 3/30
26/26 - 10s - loss: 1.5526 - accuracy: 0.3445 - val_loss: 1.5880 - val_accuracy: 0.2587 - 10s/epoch - 387ms/step
Epoch 4/30
26/26 - 10s - loss: 1.3003 - accuracy: 0.4826 - val_loss: 1.6011 - val_accuracy: 0.2687 - 10s/epoch - 388ms/step
Epoch 5/30
26/26 - 10s - loss: 0.8692 - accuracy: 0.7040 - val_loss: 1.9565 - val_accuracy: 0.3134 - 10s/epoch - 389ms/step
Epoch 6/30
26/26 - 10s - loss: 0.4638 - accuracy: 0.8321 - val_loss: 2.5081 - val_accuracy: 0.2935 - 10s/epoch - 388ms/step
Epoch 7/30
26/26 - 10s - loss: 0.2235 - accuracy: 0.9478 - val_loss: 3.2607 - val_accuracy: 0.3085 - 10s/epoch - 387ms/step
Epoch 8/30
26/26 - 10s - loss: 0.0987 - accuracy: 0.9776 - val_loss: 3.6185 - val_accuracy: 0.3284 - 10s/epoch - 382ms/step
Epoch 9/